In [1]:
import numpy as np
from pyspark import SparkContext
sc = SparkContext("local[*]", "Practica1")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 15:29:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def readFile(filename):
    text_file = sc.textFile(filename)
    text_file = text_file.map(lambda linea: np.array(linea.split(',')).astype(float))
    text_file= text_file.map(lambda x: (x[0:-1], int(x[-1])))
    return text_file    

CSV separado por comas ","

num filas= 1.000.000, num columnas= 12

Cada celda es un numero. 

La ultima celda de cada fila es la etiqueta (0/1)

FUNCIONES textFile, count, map, flatMap, reduce and reduceByKey. 

In [17]:
text_file= readFile('botnet_reduced_10k_l.csv')
text_file

PythonRDD[19] at RDD at PythonRDD.scala:53

In [18]:
def normalize (RDD_Xy):
    # ------------------------------------------------------------------------
    rdd_rows = RDD_Xy.map(lambda row: row[0])  
    sumas = rdd_rows.reduce(lambda x, y: x + y)  
    means_array = sumas / RDD_Xy.count()
    #print(f"5 Primeras medias: {means_array[0:5]}")
    
    rdd_diff = rdd_rows.map(lambda x_array: (x_array - means_array)**2)
    array_diff_sumatorio = rdd_diff.reduce(lambda x_row, y_row: x_row + y_row)
    array_varianza = np.maximum(array_diff_sumatorio / RDD_Xy.count(), 0)
    #print(f"5 Primeras Varianzas: {array_varianza[0:5]}")

    
    # rdd_final = rdd_rows.map(lambda row: np.array([(value - means_array[idx_column]) / array_varianza[idx_column] for idx_column, value in enumerate(row)]))
    # #rdd_final.collect())
    # print(rdd_final.collect())

    rdd_final = RDD_Xy.map(lambda row: (
        np.array([(value - means_array[idx_column]) / np.sqrt(array_varianza[idx_column]) 
                  for idx_column, value in enumerate(row[0])]), row[1]
    ) )
    return rdd_final
    #print(rdd_final.collect())

In [19]:
rdd1 = normalize(text_file)
rdd1.take(2)

[(array([ 1.41281668, -0.75956071, -0.41204268, -0.45940748,  1.39580934,
         -0.35386218,  0.74057161, -0.8945781 , -0.40163083, -2.91039199,
          0.1567098 ]),
  0),
 (array([-0.79867306, -0.89146231,  3.64034447, -0.45940748, -0.52296869,
         -0.35387136,  0.55184122, -0.8945781 , -1.27444226,  0.4762026 ,
          0.1567098 ]),
  1)]

In [20]:
rdd6= rdd1.map(lambda x : x[0])
rdd6.stdev()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [21]:
#rdd_weighted_sum = rdd.map(lambda row: sum(w * x for w, x in zip(weights, row)))

In [24]:
def compute_sigmoid(z):
    y = (1 /(1 + np.exp(-z)))
    return y  
    
def train(RDD_Xy, iterations, learning_rate, lamba_reg):
    #inicializar pesos
    weights= np.random.rand(11)
    biases = np.random.rand(1)   
    #print((weights, biases))

    for it in range(iterations):
        #número de datos
        m=RDD_Xy.count()
        #calcular predicción (aplicar sigmoide a la multiplicación de los pesos más el sesgi)
        rdd2=RDD_Xy.map(lambda x: (x[0], x[1],compute_sigmoid(np.dot(x[0],weights) + biases)))

        item_2_costFunction = (lamba_reg/(2*11))*np.array([w**2 for w in weights]).sum() 
        # print(f"Calculo 2: {item_2_costFunction}")
        
        rdd2 = rdd2.map(lambda x: (x[0], x[1], x[2], -(x[1]*np.log(x[2]) + (1 - x[1])*np.log(1-x[2]))))
        coste=rdd2.map(lambda x:x[3] )
        coste=coste.reduce(lambda x,y:x+y )
        
        #fórmula de la derivada
        rdd3= rdd2.map(lambda x: ((x[2]-x[1])*x[0],(x[2]-x[1])))
        rdd4= rdd3.reduce(lambda x,y: (x[0]+y[0], x[1]+y[1]))
        weights2 = (weights * lamba_reg)/ len(weights)
        d_weights= ((rdd4[0]/ m) + weights2 ) 
        d_bias= -(rdd4[1]) / m
        #actualización de las derivadas
        weights = weights - learning_rate * d_weights
        biases = biases - learning_rate * d_bias


        coste=(coste/m) +  item_2_costFunction
        print(f"Iteration [{it}]: {coste}")
                
    return weights, biases

train(rdd1, 10, 1.5, 1)
    

Iteration [0]: [1.35588688]


Iteration [1]: [0.68341465]


Iteration [2]: [0.53261946]


Iteration [3]: [0.50593201]


Iteration [4]: [0.52400947]


Iteration [5]: [0.56981694]


Iteration [6]: [0.64001382]


Iteration [7]: [0.73482541]
Iteration [8]: [0.8548204]


Iteration [9]: [0.99972771]


(array([-0.28405648, -0.76312973, -0.32437063, -0.14680474, -0.86540707,
         0.84909936,  0.04750503, -1.13246116,  0.40680193,  0.59260817,
         0.48080703]),
 array([3.70002409]))

In [25]:
def predict(w, b, X):
    y = compute_sigmoid(np.dot(X,w) + b)
    if y >= 0.5:
        return 1
    else: 
        return 0
    

In [26]:
def accuracy(w,b, RDD_Xy):
    y_hat = RDD_Xy.map(lambda x: (predict(w, b, x[0]), x[1]))  
    preds = y_hat.map(lambda x: 1 if x[0] == x[1] else 0)
    correct_preds = preds.reduce(lambda x, y: x + y)    
    total = RDD_Xy.count()
    accuracy = correct_preds / total
    
    return accuracy


In [27]:
# TEST
weights, biases = train(rdd1, 10, 1.5, 0)
acc = accuracy(weights, biases, rdd1)
acc

Iteration [0]: [1.20318655]


Iteration [1]: [0.75198078]
Iteration [2]: [0.58607128]


Iteration [3]: [0.53033]


Iteration [4]: [0.52185114]


Iteration [5]: [0.53880943]


Iteration [6]: [0.57233966]


Iteration [7]: [0.61803449]


Iteration [8]: [0.67322985]


Iteration [9]: [0.73605681]


0.6894

In [28]:
# read data
path = 'botnet_reduced_10k_l.csv'
data = readFile(path)

# standardize
data = normalize(data)

nIter = 10
learningRate = 1.5
lamba_reg = 0

weights, bias = train(data, nIter, learningRate, lamba_reg)
acc = accuracy(weights, bias, data)
print ("acc", acc)

Iteration [0]: [1.0388845]


Iteration [1]: [0.48985204]
Iteration [2]: [0.35429785]


Iteration [3]: [0.31408856]


Iteration [4]: [0.29791856]


Iteration [5]: [0.29243179]


Iteration [6]: [0.29381734]
Iteration [7]: [0.30075361]
Iteration [8]: [0.31282163]


Iteration [9]: [0.32999612]


acc 0.8383


### Ejercicio 2

In [29]:
import random
def transform(data, m, n_blocks):    

    m_per_block = m / n_blocks
    
    indices = list(range(m)) 
    random.shuffle(indices)

    indexed_data = data.map(lambda x: (1, [x]))
    indexed_data = indexed_data.reduceByKey(lambda a, b: a + b) 
    indexed_data = indexed_data.flatMap(lambda x: [(indices[i]//m_per_block, item) for i, item in enumerate(x[1])])

    return indexed_data

In [30]:
import random
def transform(data, m, n_blocks):   
    #calcular n de elementos de cada bloque
    #'//' para obtener un valor entero (división entera) sin decimales
    m_per_block = m // n_blocks

    #creamos una lista de índices que representan las filas del conjunto de datos
    #tenemos lista tiene valores del 0 al m-1 (m es el número total de filas)
    indices = list(range(m)) 
    # mezzclamos los índices para asegurarnos de que los datos se distribuyan de forma aleatoria entre los bloques
    random.shuffle(indices)

    #la operación modulo % asegura que el índice de cada fila se asigna a un bloque específico.
    # el resultado de 'indices.pop() % n_blocks' será un valor entre 0 y (n_blocks-1)
    #el operados hace que los datos se distribuyan entre los bloques de manera UNIFORME
    # los datos se asignen cíclicamente a bloques (0, 1, 2,... hasta n-1), y como esta desordenado ya se aplica el shuffle.


    #ahora se tiene el resultado de la operacion modulo (y se añade como clave a cada fila)
    indexed_data = data.map(lambda x: (indices.pop() % n_blocks, x)) 
    #'x' es una fila del RDD original, y 'indices.pop() % n_blocks' le asigna un bloque aleatorio 
    #cada bloque tiene mas o menos el mismo número de elementos

    #se puede comprobar con    
    conteo_rdd = indexed_data.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)

    #agrupar por bloque y hacer cuenta del numero de elementos de ese bloque
    for bloque, count in conteo_rdd.collect():
        print(f"Bloque {bloque}: {count} elementos")

    #esto lo he copiado
    return indexed_data.cache()


In [31]:
#pruebas 

path = 'botnet_reduced_10k_l.csv'
data = readFile(path)

# standardize
data = normalize(data)
num_blocks_cv = 3
data_cv = transform(data, data.count(), num_blocks_cv)


Bloque 2: 3324 elementos
Bloque 0: 3326 elementos
Bloque 1: 3350 elementos


In [32]:
data_cv.take(3)

[(2,
  (array([ 1.41281668, -0.75956071, -0.41204268, -0.45940748,  1.39580934,
          -0.35386218,  0.74057161, -0.8945781 , -0.40163083, -2.91039199,
           0.1567098 ]),
   0)),
 (1,
  (array([-0.79867306, -0.89146231,  3.64034447, -0.45940748, -0.52296869,
          -0.35387136,  0.55184122, -0.8945781 , -1.27444226,  0.4762026 ,
           0.1567098 ]),
   1)),
 (2,
  (array([ 1.44693571,  1.38000803, -0.41371517, -0.45940748, -0.52296783,
          -0.35387107,  0.74057161,  1.49057358,  0.9904482 , -0.73801555,
          -1.74828175]),
   0))]

In [33]:
def get_block_data(data_cv, i, n_blocks):
    #agrupamos los datos por bloque usando el valor de la clave (n de bloque) y almacenamos los datos al lado
    #tengo (bloque/slice , dato (array y etiqueta))
    grouped_data = data_cv.map(lambda x: (x[0], [x[1]])) 

    
    #reduceByKey' para concatenar los elementos de la misma clave (bloque), es decir, agrupamos
    #los datos que tienen el mismo valor de bloque.
    grouped_data = grouped_data.reduceByKey(lambda a, b: a + b) 
    #aqui tenemos clave (n de bloque, lista de datos (filas con su array de 11 elementos, etiqueta))
    
    #en cada iteracion de este bucle trabajo con un slice o bloque 
    #tomamos todos los elementos cuyo bloque no sea el valor 'i' actual.
    tr_data = grouped_data.map(lambda x: (x[0], [item for item in x[1] if x[0] != i]))
    test_data = grouped_data.map(lambda x: (x[0], [item for item in x[1] if x[0] == i])) # el resto de test (elementos con clave el i)

    # hasta ahora tenia lista de listas para cada bloque (al agrupar con reducebykey)
    #aplano y tengo solo los datos no la clave del bloque (ya no se va a usar)
    tr_data = tr_data.flatMap(lambda x: x[1])  
    test_data = test_data.flatMap(lambda x: x[1])  
    
    return tr_data, test_data
    return tr_data, test_data

In [34]:
# read data
path = 'botnet_reduced_10k_l.csv'
data = readFile(path)

# standardize
data = normalize(data)

num_blocks_cv = 3

nIter = 10
learningRate = 1.5
lamba_reg = 0

accuracies = []

for i in range(num_blocks_cv):
    tr_data, test_data = get_block_data(data_cv, i, num_blocks_cv)    
    print(f"\nSlice {i+1}:")
    print(f"  Datos de Entrenamiento - Número de elementos: {tr_data.count()}")
    print(f"  Datos de Prueba - Número de elementos: {test_data.count()}")
    #el profe en el enunciado devuelve solo ws, la funcion creada antes devuelve w, b separadamente
    w,b = train(tr_data,nIter,learningRate, lamba_reg)
    #el profe en el enunciado coge dos argumentos, en la funcion creada antes se cogen 3
    acc = accuracy(w,b, test_data)
    print("acc :", acc)
    #guardar acc de cada iteracion (que es con distinto conjunto de datos de train y test)
    accuracies.append(acc)

avg_acc = sum(accuracies) / num_blocks_cv
print("average acc: ", avg_acc)
    


Slice 1:


  Datos de Entrenamiento - Número de elementos: 6674
  Datos de Prueba - Número de elementos: 3326
Iteration [0]: [1.52733507]
Iteration [1]: [0.76857992]
Iteration [2]: [0.49949568]
Iteration [3]: [0.40029288]
Iteration [4]: [0.35544898]
Iteration [5]: [0.33358647]
Iteration [6]: [0.32456327]
Iteration [7]: [0.32448411]
Iteration [8]: [0.3317352]


Iteration [9]: [0.34568371]
acc : 0.8268190018039687

Slice 2:
  Datos de Entrenamiento - Número de elementos: 6650
  Datos de Prueba - Número de elementos: 3350
Iteration [0]: [1.33722647]
Iteration [1]: [0.71470643]


Iteration [2]: [0.49151176]
Iteration [3]: [0.40411675]
Iteration [4]: [0.36687761]
Iteration [5]: [0.35404026]
Iteration [6]: [0.35595065]
Iteration [7]: [0.36854262]
Iteration [8]: [0.38997965]
Iteration [9]: [0.41936219]
acc : 0.7862686567164179

Slice 3:
  Datos de Entrenamiento - Número de elementos: 6676
  Datos de Prueba - Número de elementos: 3324
Iteration [0]: [1.56012231]
Iteration [1]: [0.73893323]
Iteration [2]: [0.43738647]
Iteration [3]: [0.34299072]
Iteration [4]: [0.3074498]
Iteration [5]: [0.29312275]
Iteration [6]: [0.28937742]
Iteration [7]: [0.29255038]
Iteration [8]: [0.30126723]
Iteration [9]: [0.31506313]
acc : 0.8423586040914561
average acc:  0.8184820875372809
